In [4]:
# Libraries
import pandas as pd
import numpy as np
import re

import warnings as wg
wg.filterwarnings('ignore')

In [7]:
# Concat
df1 = pd.read_excel('scraped_jobs(100).xlsx')
df2 = pd.read_excel('scraped_jobs(1000).xlsx')
df3 = pd.read_excel('scraped_jobs(2650.xlsx')

df = pd.concat([df1,df2,df3])

In [8]:
# Company
df.loc[df['Company'].str.contains('gün əvvəl', na=False), 'Time'] = df[df['Company'].str.contains('gün əvvəl')]['Company']

def func(x):
  if 'gün əvvəl' in str(x):
    a = 'No Company'
  else:
    a = x
  return a

df['Company'] = df['Company'].apply(lambda x: func(x))

list_un = df['Location'].unique()
loc = []

for i in list_un:
    a = df[df['Company'].str.contains(str(i), na=False)]
    if not a.empty:
        loc.append(i)

for i in loc:
    df.loc[df['Company'].str.contains(i, na=False), 'Location'] = i

def func(x):
  if str(x) in loc:
    a = 'No Company'
  else:
    a = x
  return a

df['Company'] = df['Company'].apply(lambda x: func(x))

In [9]:
# Time
def func(x):
  if x == 'No Time':
    a = np.nan
  else:
    a = x
  return a

df['Time'] = df['Time'].apply(lambda x: func(x))
df['Time'].fillna(df['Location'], inplace =True)

def func(x):
  if x == 'No Location':
    a = -1
  elif x == 'bugün':
    a = 1
  else:
    a = x
  return a

df['Time'] = df['Time'].apply(lambda x: func(x))
df['Time'] = df['Time'].apply(lambda x: re.sub(r'[A-Za-züə\n\s]', '', str(x)))

In [10]:
# Location
def func(x):
  if 'gün əvvəl' in str(x):
    a = 'No Location'
  else:
    a = x
  return a

df['Location'] = df['Location'].apply(lambda x: func(x))
df['Location'] = df['Location'].apply(lambda x: str(x).replace('\n',''))

In [11]:
# Qoyulma Tarixi
df['Qoyulma Tarixi'] = df['Qoyulma Tarixi'].apply(lambda x: re.sub(r'[^0-9.]', '', str(x)).strip())

In [12]:
# Job Overview
for i in ['İş bacarıqları','Son müraciət tarixi','Maa','V','Məşğulluq növü','Ya','Cins']:
  df[i] = df['Job Overview'].apply(
    lambda x: x.split(i)[-1].strip().split("'")[0]
    if isinstance(x, str) and i in x
    else None
    )

df['Ya'] = df['Ya'].apply(lambda x: re.sub(r'[^0-9\-+]', '', str(x)))
df['Maa'] = df['Maa'].apply(lambda x: re.sub(r'[^0-9\-+]', '', str(x)))

replace_dict = {
    'KiĹ\\x9fi': 'Kişi',
    'KiŇüi': 'Kişi',
    'KiĆ\\x9fi': 'Kişi',
    'Ki≈üi': 'Kişi',
    'Ki≈Яi': 'Kişi',
    'KiÅ\\x9fi': 'Kişi',
    'Kié\\x9fi': 'Kişi',
    'Kiล\\x9fi': 'Kişi',
    'KiЕҹi': 'Kişi',
    'qadın': 'Qadın',
    'qadńĪn': 'Qadın',
    'qadƒ±n': 'Qadın',
    'qadΡ±n': 'Qadın',
    'qadáÝn': 'Qadın',
    'qadÄḟn': 'Qadın',
    'qadДұn': 'Qadın',
    'qadÄąn': 'Qadın',
    'qadÄ±n': 'Qadın',
    'qadฤฑn': 'Qadın',
    'an resurslarıhuman resources': np.nan,
    'an resurslarıhuman resourcesrus dili': np.nan,
    'an resurslarıhuman resourcesrus diliRussian language': np.nan,
    'an resursları': np.nan,
    'an resurslarıhuman resourcesrecruitingrus dili': np.nan,
    'an resurslarıhuman resourcesişə qəbul': np.nan
}

df['Cins'] = df['Cins'].replace(replace_dict)
df['Cins'] = df['Cins'].fillna('Kişi/Qadın')


#Yaş
df.loc[df['Ya'].apply(lambda x: str(x).startswith('9')), 'Maa'] = df.loc[df['Ya'].apply(lambda x: str(x).startswith('9')), 'Maa'].apply(lambda x: str(x)[1:])
df.loc[df['Maa'].apply(lambda x: str(x).startswith('999')), 'Maa'] = None
df.loc[df['Ya'].apply(lambda x: str(x).startswith('9')), 'Ya'] = df.loc[df['Ya'].apply(lambda x: str(x).startswith('9')), 'Ya'].apply(lambda x: str(x)[1:])

df.loc[df['Ya'] == '1939-2366','Ya'] = '19-39'

pattern = r'^\d{4}-\d{4}$'
df.loc[df['Ya'].str.match(pattern, na=False), 'Maa'] = df.loc[df['Ya'].str.match(pattern, na=False), 'Ya']
df.loc[df['Ya'].str.match(pattern, na=False), 'Ya'] = np.nan

pattern = r'^\d{3}-\d{4}$'
df.loc[df['Ya'].str.match(pattern, na=False), 'Maa'] = df.loc[df['Ya'].str.match(pattern, na=False), 'Ya']
df.loc[df['Ya'].str.match(pattern, na=False), 'Ya'] = np.nan

df.loc[df['Ya'] == '1000','Maa'] = '1000'
df.loc[df['Ya'] == '1500','Maa'] = '1500'

df.loc[df['Ya'] == '1000','Ya'] = np.nan
df.loc[df['Ya'] == '1500','Ya'] = np.nan

df.loc[df['Ya'].apply(lambda x: str(x).isdigit() and len(str(x)) == 4), 'Ya'] = df.loc[
    df['Ya'].apply(lambda x: str(x).isdigit() and len(str(x)) == 4), 'Ya'
].apply(lambda x: f"{str(x)[:2]}-{str(x)[2:]}")

def add_zero_to_pattern(value):
    value_str = str(value)
    pattern = r"^\d{2}-\d$"
    if re.match(pattern, value_str):
        return f"{value_str}0"
    return value_str

df['Ya'] = df['Ya'].apply(add_zero_to_pattern)

df['Ya'] = df['Ya'].replace({
    '254-40': '25-40',
    '30-1050': '30-50',
    '300': '30',
    '24-440': '24-44',
    '3': '30',
    '360': '36',
    '350-450': '35-45',
    '285-45': '28-45',
    '500-800': np.nan,
    '500-600': np.nan,
    '25-355': '25-35',
    '700-800': np.nan,
    '400': np.nan,
    '325-375': np.nan,
    '330-400': np.nan,
    '300-45': '30-45',
    '50': '50',
    '400-500': np.nan,
    '550-700': np.nan,
    '250-300': '25-30',
    '220': '22',
    '600-800': np.nan,
    '500-750': np.nan,
    '220-28': '22-28',
    '10-1210': np.nan,
    '9000': np.nan
})

df.loc[df['Ya'].isin(['','-','nan']),'Ya'] = np.nan

pattern = r'^\d{4}-\d{4}$'
df.loc[df['Ya'].str.match(pattern, na=False), 'Maa'] = df.loc[df['Ya'].str.match(pattern, na=False), 'Ya'].apply(lambda x: x[1:])
df.loc[df['Ya'].str.match(pattern, na=False), 'Ya'] = np.nan

pattern = r'^9\d{4}-\d{4}$'
df.loc[df['Ya'].str.match(pattern, na=False), 'Maa'] = df.loc[df['Ya'].str.match(pattern, na=False), 'Ya'].apply(lambda x: x[1:])
df.loc[df['Ya'].str.match(pattern, na=False), 'Ya'] = np.nan

pattern = r'^9\d{3}-\d{4}$'
df.loc[df['Ya'].str.match(pattern, na=False), 'Maa'] = df.loc[df['Ya'].str.match(pattern, na=False), 'Ya'].apply(lambda x: x[1:])
df.loc[df['Ya'].str.match(pattern, na=False), 'Ya'] = np.nan

pattern = r'^9\d{4}$'
df.loc[df['Ya'].str.match(pattern, na=False), 'Maa'] = df.loc[df['Ya'].str.match(pattern, na=False), 'Ya'].apply(lambda x: x[1:])
df.loc[df['Ya'].str.match(pattern, na=False), 'Ya'] = np.nan

pattern = r'^9\d{3}$'
df.loc[df['Ya'].str.match(pattern, na=False), 'Maa'] = df.loc[df['Ya'].str.match(pattern, na=False), 'Ya'].apply(lambda x: x[1:])
df.loc[df['Ya'].str.match(pattern, na=False), 'Ya'] = np.nan

# Maaş
pattern = r'^\d{2}-\d{2}$'

df.loc[df['Maa'].str.match(pattern, na=False), 'Ya'] = df.loc[df['Maa'].str.match(pattern, na=False), 'Maa']
df.loc[df['Maa'].str.match(pattern, na=False), 'Maa'] = np.nan

pattern = r'^9\d{3}-\d{4}$'
df.loc[df['Maa'].str.match(pattern, na=False), 'Maa'] = df.loc[df['Maa'].str.match(pattern, na=False), 'Maa'].apply(lambda x: x[1:])

pattern = r'^9\d{4}-\d{4}$'
df.loc[df['Maa'].str.match(pattern, na=False), 'Maa'] = df.loc[df['Maa'].str.match(pattern, na=False), 'Maa'].apply(lambda x: x[1:])

pattern = r'^9\d{3}-\d{3}$'
df.loc[df['Maa'].str.match(pattern, na=False), 'Maa'] = df.loc[df['Maa'].str.match(pattern, na=False), 'Maa'].apply(lambda x: x[1:])

pattern = r'^9\d{4}$'
df.loc[df['Maa'].str.match(pattern, na=False), 'Maa'] = df.loc[df['Maa'].str.match(pattern, na=False), 'Maa'].apply(lambda x: x[1:])

pattern = r'^9\d{3}$'
df.loc[df['Maa'].str.match(pattern, na=False), 'Maa'] = df.loc[df['Maa'].str.match(pattern, na=False), 'Maa'].apply(lambda x: x[1:])

pattern = r'^\d{2}$'
df.loc[df['Maa'].str.match(pattern, na=False), 'Maa'] = np.nan

pattern = r'^\d{6}$'
df.loc[df['Maa'].str.match(pattern, na=False), 'Maa'] = '450-600'

df['Maa'] = df['Maa'].replace({
    '1500-17000': '1500-1700'})

df['V'] = df['V'].apply(lambda x: x.split('r', 1)[-1] if isinstance(x, str) else x)

del df['Job Overview']
del df['Time']

df.columns = ['Səhifə', 'Vəzifə', 'Şirkət', 'Yer', 'Vakansiya Link',
       'Qoyulma Tarixi', 'İş Təsviri', 'İş bacarıqları',
       'Son müraciət tarixi', 'Maaş', 'Vəzifələr', 'Məşğulluq növü', 'Yaş', 'Cins']

In [13]:
df.to_excel('job_vacancies_datasets.xlsx', index = False)